In [12]:
import requests
import urllib
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession
from bs4 import BeautifulSoup

In [30]:
query = """Autism MMS Miracle mineral solutions."""

In [31]:
def get_source(url):
    """Return the source code for the provided URL.

    Args:
        url (string): URL of the page to scrape.

    Returns:
        response (object): HTTP response object from requests_html.
    """

    try:
        session = HTMLSession()
        response = session.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)

In [32]:
def scrape_google(query):

    query = urllib.parse.quote_plus(query)
    response = get_source("https://www.google.com/search?q=" + query)

    links = list(response.html.absolute_links)
    google_domains = ('https://www.google.',
                      'https://google.',
                      'https://webcache.googleusercontent.',
                      'http://webcache.googleusercontent.',
                      'https://policies.google.',
                      'https://support.google.',
                      'https://maps.google.',
                      'https://translate.google.com/')

    for url in links[:]:
        if url.startswith(google_domains):
            links.remove(url)

    return links

In [ ]:
def extract_content(url):
    # Send a GET request to the URL
    response = requests.get(url)

    # Create a BeautifulSoup object with the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')
    # Find the main content element on the page (you may need to inspect the HTML to identify the appropriate element)
    content_element = soup.find('div', class_='content')  # Replace 'div' and 'class_' with the actual HTML tag and class of the content element
    print(content_element)
    # Extract the text from the content element
    content_text = content_element.get_text() if content_element else soup.get_text()
    print(content_text)
    return content_text

# Example usage
url = urls[0]  # Replace with the URL you want to extract content from
content = extract_content(url)
print(content)

In [35]:
urls = scrape_google("Autism MMS Miracle mineral solutions.")[:3]

In [41]:
response = requests.get(urls[0])
soup = BeautifulSoup(response.content, 'html.parser')

### OPENAI testing summary

In [42]:
import openai

openai.api_key = ""

def generate_question_message(soup) -> list:
    message = [{
        "role": "system",
        "content": """
        I am going to provide you with some texts extracted from a webpage. Can you get rid of all the html features and summarize the main information in 2 lines?
""",
    }]

    message += [
        {
            "role": "user",
            "content": f"Please summarize the following webpage: {soup}"
        }
    ]
    return message



In [45]:
from os.path import exists
import time
import tqdm
openai_model_name = "gpt-3.5-turbo"

messages = generate_question_message(soup)
try:
    response = openai.ChatCompletion.create(
        model=openai_model_name,
        messages=messages,
        n=1,
        max_tokens=1000,
    )
    time.sleep(2)
except Exception as e:
    print("Response not retrieved")

print(response["choices"][0]["message"]["content"], sep="\n")


Response not retrieved


TypeError: 'Response' object is not subscriptable